<a href="https://colab.research.google.com/github/greengerong/awesome-llm/blob/main/colab/green_starchat_alpha_%E6%A8%A1%E5%9E%8B%E5%BE%AE%E8%B0%83.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# starchat-alpha微调

参考：[https://huggingface.co/blog/zh/starchat-alpha](https://huggingface.co/blog/zh/starchat-alpha)

In [ ]:
!pip install datasets  transformers huggingface

# 数据集处理

In [ ]:
from datasets import load_dataset

dataset = load_dataset("HuggingFaceH4/oasst1_en")
print(dataset)
print("-------------------")
sample = dataset["train_ift"][0]
print(sample)

一个更好的方法是使用一种结构化的格式，比如 ChatML。这种格式会对每一个对话轮次进行包装。包装使用的是一些特殊的 token，用以标明询问或回答的角色。

在这种格式下，我们使用这些特殊的 token:

<|system|>: 表示系统信息开始的地方，这里的系统信息描述了这个聊天机器人的身份角色。
<|user|>: 表示这里的话语是人类用户说出来的。
<|assistant|>: 表示这里的话语是 AI 机器人说出来的。
<|end|>: 表示说话内容的结尾，或系统信息的结尾。
下面我们写一个函数，把我们的实例数据用这些特殊的 token 包装起来:

In [ ]:
system_token = "<|assistant|>"
user_token = "<|user|>"
assistant_token = "<|assistant|>"
end_token = "<|end|>"

def prepare_dialogue(example):
    system_msg = "Below is a dialogue between a human and an AI assistant called StarChat."
    prompt = system_token + "\n" + system_msg + end_token + "\n"
    for message in example["messages"]:
        if message["role"] == "user":
            prompt += user_token + "\n" + message["content"] + end_token + "\n"
        else:
            prompt += assistant_token + "\n" + message["content"] + end_token + "\n"
    return prompt

print(prepare_dialogue(sample))


以上就是包装好后的数据！下一步，我们还需要把这些特殊的 token 加入到分词器 (tokenizer) 的词汇表中。我们这里下载 StarCoder 的分词器，然后加入这些特殊 token:

In [ ]:
!huggingface-cli login

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("bigcode/starcoderbase")
tokenizer.add_special_tokens({"additional_special_tokens": ["<|system|>", "<|assistant|>", "<|user|>", "<|end|>"]})
# Check the tokens have been added
print(tokenizer.special_tokens_map)
print("-------------------")
print(tokenizer("<|assistant|>"))

掩盖掉用户话语部分的标签

In [ ]:
def mask_user_labels(tokenizer, labels):
    user_token_id = tokenizer.convert_tokens_to_ids(user_token)
    assistant_token_id = tokenizer.convert_tokens_to_ids(assistant_token)
    for idx, label_id in enumerate(labels):
        if label_id == user_token_id:
            current_idx = idx
            while labels[current_idx]!= assistant_token_id and current_idx < len(labels):
                labels[current_idx] = -100 # Ignored by the loss
                current_idx += 1

dialogue = "<|user|>\nHello, can you help me?<|end|>\n<|assistant|>\nSure, what can I do for you?<|end|>\n"
input_ids = tokenizer(dialogue).input_ids
labels = input_ids.copy()
mask_user_labels(tokenizer, labels)
print(labels)


# 使用 DeepSpeed ZeRO-3 微调 StarCoder

In [ ]:
!rm -rf starcoder
!git clone https://github.com/greengerong/starcoder.git
!pip install -r /content/starcoder/chat/requirements.txt
!sudo apt-get install git-lfs
!pip3 install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118
!pip install -U deepspeed

接下来我们就可以训练了

In [ ]:
!torchrun  /content/starcoder/chat/train.py /content/starcoder/chat/config.yaml --deepspeed=/content/starcoder/chat/deepspeed_z3_config_bf16.json